In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
%matplotlib inline
import gc
import os 
from time import time
import sys
from sklearn.model_selection import KFold, GroupKFold, train_test_split
import json
from numba import jit
import re

from abc import ABCMeta, abstractmethod
from pathlib import Path

import scipy
import random
from joblib import Parallel, delayed
import multiprocessing

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error
import pickle

# import modules
# sys.path.append('../utils/')
# from large_file_pickle import pickle_dump, pickle_load
# # from notifications import send_line_notification, notify_slack
# from memory_optimize import memory_reducer
# from util import *

import logging
import time
from typing import List, Optional, Union, Tuple, Dict
# from encoders import frequency_encoding
from contextlib import contextmanager

sys.path.append('../py/')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 1000)


# modules  



## utils

### util function


In [ ]:
def groupings(df, cols, agg_dict, pref='') -> object:
    """
    Returns:
        object: 
    """
    group_df = df.groupby(cols).agg(agg_dict)
    group_df.columns = [pref + c[0] + "_" + c[1] for c in list(group_df.columns)]
    group_df.reset_index(inplace = True)
    
    return group_df

@contextmanager
def timer(name, logger=None):
    """時間計測
    """
    t0 = time.time()
    if logger:
        logger.log(logging.DEBUG, f'[{name}] start')
    else:
        print(f'[{name}] start')
    yield
    if logger:
        logger.log(logging.DEBUG, f'[{name}] done in {time.time() - t0:.0f} s')
    else:
        print(f'[{name}] done in {time.time() - t0:.0f} s')

def get_val_score(y_true, y_pred, obj="RMSE"):
    # RMSE
    if obj == "RMSE":
        val_score = np.sqrt(mean_squared_error(y_true, y_pred))
    elif obj == "QWK":
        val_score = qwk(y_true, y_pred, max_rat=3)
    else:
        raise ValueError("valuation is not defined!")
    return val_score

def memory_reducer(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        print(col)
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def applyParallel(dfGrouped, func):
    """関数の並列処理
    """
    retLst = Parallel(n_jobs=multiprocessing.cpu_count(), verbose=5)(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)
        


### logs 

In [ ]:
from lightgbm.callback import _format_eval_result

def log_evaluation(logger, period=100, show_stdv=True, level=logging.DEBUG):
    def _callback(env):
        if period > 0 and env.evaluation_result_list and (env.iteration + 1) % period == 0:
            result = '\t'.join([_format_eval_result(x, show_stdv) for x in env.evaluation_result_list])
            logger.log(level, '[{}]\t{}'.format(env.iteration + 1, result))

    _callback.order = 10
    return _callback

# ロガーの作成
# logging.basicConfig(level=logging.DEBUG)

def log_output(subject):
    logger = logging.getLogger('main')
    for h in logger.handlers:
        logger.removeHandler(h)

    logger.setLevel(logging.DEBUG)
    now = int(time.time())

    log_dir = os.path.join(os.path.dirname("__file__"), "../logs")
    os.makedirs(log_dir, exist_ok=True)

    log_path = Path(log_dir) / "{}_{}.log".format(subject, now)
    fh = logging.FileHandler(log_path)
    logger.addHandler(fh)

    return logger, log_path

### pickle files

In [ ]:

class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)

        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            # print(n, idx)
            batch_size = min(n - idx, 1 << 31 - 1)
            # print(batch_size)
            # print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            # print("done.", flush=True)
            idx += batch_size
        print("calculate done!")


def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

## Feature class

In [ ]:
class Features(metaclass=ABCMeta):

    def __init__(self, params, logger=None):
        self.name = self.__class__.__name__
        self.datatype = params["datatype"]
        self.debug = params["debug"]
        self.is_overwrite = params["is_overwrite"]
        self.org_columns = []
        self.logger = logger

        self.input_dir = os.path.join(os.path.dirname("__file__"), "../input")
        self.df_path = Path(self.input_dir) / f"{self.datatype}.csv"

        self.save_dir = os.path.join(os.path.dirname("__file__"), f"../feature")
        self.save_type_dir = Path(self.save_dir) / f"{self.datatype}"
        self.save_path = Path(self.save_type_dir) / f"{self.name}.pkl"
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(self.save_type_dir, exist_ok=True)

    def feature_extract(self, org_train, org_test):
        if self.check_feature_exec():
            with timer(f"FE: {self.name}", self.logger):
                a = self.calc_feature(org_train, org_test)
            return a

    @abstractmethod
    def calc_feature(self):
        """calc and save features
        Return: feature_df
        """
        raise NotImplementedError

    def format_and_save_feats(self, feat_df):
        """保存するカラムなど特徴量の形式を指定する
        """
        feat_cols = [c for c in list(feat_df.columns) if c not in self.org_columns]
        pickle_dump(feat_df[feat_cols], self.save_path)

        del feat_df
        gc.collect()

    def check_feature_exec(self):
        """
        すでに対象の特徴が存在するかどうかをcheckする
        Returns: bool (Falseなら特徴作成しない)

        """
        path = self.save_path

        if self.is_overwrite:
            print(f"overwrite features : {self.name}")
            return True
        else:
            if os.path.exists(path) is False:
                print(f"creates new file : {self.name}")
                return True

        print(f"file exists : {self.name}")
        return False


## model class


In [ ]:
class LightGBM():
    def __init__(self, param):

        self.predict_type = param["predict_type"] # classifier, regressor
        self.train_params = param["train_params"]
        self.train_cols = param["train_cols"]
        self.cat_cols = param["cat_cols"]
        self.target = param["target"]
        self.is_debug = param["is_debug"]

    def train(self, train, valid, logger):
        if type(train) != pd.DataFrame or type(valid) != pd.DataFrame:
            raise ValueError('Parameter train and valid must be pandas.DataFrame')

        if list(train.columns) != list(valid.columns):
            raise ValueError('Train and valid must have a same column list')

        trn_x, trn_y = train[self.train_cols], train[self.target]
        val_x, val_y = valid[self.train_cols], valid[self.target]
        callbacks = [log_evaluation(logger, period=500)]
        
        if self.predict_type == "binary_classifier":
            clf = lgb.LGBMClassifier(**self.train_params)
            clf.fit(
                trn_x, trn_y,
                eval_set=[(trn_x, trn_y), (val_x, val_y)],
                verbose=500,
                early_stopping_rounds=500,
                callbacks=callbacks,
                categorical_feature = self.cat_cols,
            )
            oof = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
            
        elif self.predict_type == "multi_classifier":
            clf = lgb.LGBMClassifier(**self.train_params)
            clf.fit(
                trn_x, trn_y,
                eval_set=[(trn_x, trn_y), (val_x, val_y)],
                verbose=500,
                early_stopping_rounds=500,
                callbacks=callbacks,
                categorical_feature = self.cat_cols,
                eval_metric=eval_qwk_lgb
            )
            oof = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)

        elif self.predict_type == "regressor":
            clf = lgb.LGBMRegressor(**self.train_params)
            clf.fit(
                trn_x, trn_y,
                eval_set=[(trn_x, trn_y), (val_x, val_y)],
                verbose=500,
                early_stopping_rounds=500,
                callbacks=callbacks,
                categorical_feature=self.cat_cols,
            )

            oof = clf.predict(val_x, num_iteration=clf.best_iteration_)

        else:
            raise ValueError("unknown prediction type !!")

        self.clf = clf

        # feature importance
        feature_importance_df = pd.DataFrame()
        feature_importance_df["feature"] = self.train_cols
        feature_importance_df["importance"] = self.clf.feature_importances_

        return clf, oof, feature_importance_df

    def predict(self, test, logger):
        if self.predict_type == "classifier":
            prediction = self.clf.predict_proba(test[self.train_cols],
                                                num_iteration=self.clf.best_iteration_)[:, 1]
        elif self.predict_type == "multi_classifier":
            prediction = self.clf.predict_proba(test[self.train_cols],
                                                num_iteration=self.clf.best_iteration_)
        elif self.predict_type == "regressor":
            prediction = self.clf.predict(test[self.train_cols],
                                          num_iteration=self.clf.best_iteration_)
        else:
            raise ValueError("unknown prediction type !!")

        return prediction

    def save_model(self, save_dir):
        pass

## validation

### QWK

In [ ]:
@jit
def qwk(y_true, y_pred, max_rat= 3):
    y_true_ = np.asarray(y_true, dtype=int)
    y_pred_ = np.asarray(y_pred, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    numerator = 0
    for k in range(y_true_.shape[0]):
        i, j = y_true_[k], y_pred_[k]
        hist1[i] += 1
        hist2[j] += 1
        numerator += (i - j) * (i - j)

    denominator = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            denominator += hist1[i] * hist2[j] * (i - j) * (i - j)

    denominator /= y_true_.shape[0]
    return 1 - numerator / denominator

def eval_qwk_lgb(y_true: Union[np.ndarray, list],
                           y_pred: Union[np.ndarray, list],) -> Tuple[str, float, bool]:
    y_pred = y_pred.reshape(len(np.unique(y_true)), -1).argmax(axis=0)
    return "qwk", qwk(y_true, y_pred), True

### define validation class

In [ ]:
class Validation():
    def __init__(self, validation_param, exp_conf, train, test, logger):
        self.model_name = validation_param["model_name"]
        self.train_small_dataset = exp_conf["train_small_dataset"]
#         self.common_conf = common_conf
        self.logger = logger
        self.exp_conf = exp_conf
        self.train = self.fix_train_size(train)
        self.test = test
        self.feature_importance = []

        self.logging_valid_parameters()

    def logging_valid_parameters(self):
#         self.logger.log(logging.DEBUG, self.common_conf)
        self.logger.log(logging.DEBUG, "[use_feature] " + "-" * 50)
        self.logger.log(logging.DEBUG, self.exp_conf["use_feature"])
        self.logger.log(logging.DEBUG, "[train_params] " + "-"*50)
        self.logger.log(logging.DEBUG, self.exp_conf["train_params"])

    def fix_train_size(self, train):
        if self.train_small_dataset:
            self.logger.log(logging.DEBUG, "Down-sampling train data.")
            self.logger.log(logging.DEBUG, f"Org-shape:{train.shape}")
            p = 0.15  # 学習に使用する割合
            np.random.seed(773)
            int_p = int(len(train.index.values) * p)
            sample_index = np.random.choice(train.index.values, int_p, replace=False)  # 重複なし

            train = train.loc[train.index.isin(sample_index)].reset_index(drop=True)
            self.logger.log(logging.DEBUG, f"sampled train-shape:{train.shape}")
            return train

        return train

    def generate_model(self, model_conf):
        if self.model_name == "LGBM":
            model = LightGBM(model_conf)
        else:
            raise ValueError("permitted models are [LGBM, ..., ]")
        return model

    def do_valid_kfold(self, model_conf, n_splits=5):
        sp = Splitter()
        target = model_conf["target"]
        split_x = self.train["installation_id"]
        split_y = self.train[target]
        seed = 773
        sp.get_kfold_idx(split_x, split_y, seed, n_cv=n_splits, stratified=False, group=True, pref=self.exp_conf["exp_name"])

        oof: ndarray = np.zeros((self.train.shape[0]))
        prediction = np.zeros((self.test.shape[0]))

        clf_list = []

        self.logger.log(logging.DEBUG, "[train cols] " + "-"*50)
        self.logger.log(logging.DEBUG, model_conf["train_cols"])
        self.validation_scores = []

        for i, (trn_idx, val_idx) in enumerate(sp.idx_list):
            self.logger.log(logging.DEBUG, "-" * 60)
            self.logger.log(logging.DEBUG, f"start training: {i}")

            with timer(f"fold {i}", self.logger):
                train_df, valid_df = self.train.loc[trn_idx], self.train.loc[val_idx]
                model = self.generate_model(model_conf)
                clf, fold_oof, feature_importance_df = model.train(train_df, valid_df, self.logger)
#                 fold_oof_class = fold_oof.argmax(axis = 1)
                
                fold_prediction = model.predict(self.test, self.logger)
#                 fold_val_score = get_val_score(valid_df[target], fold_oof_class, "QWK")

                # calc validation score using best iteration
#                 self.validation_scores.append(fold_val_score)
#                 self.logger.log(logging.DEBUG, f"fold_val_score: {fold_val_score:,.5f}")
                
                clf_list.append(clf)
                oof[val_idx] = fold_oof

                prediction += fold_prediction / n_splits

                feature_importance_df["fold"] = i
                self.feature_importance.append(feature_importance_df)

#         self.logger.log(logging.DEBUG,
#                         f"Total Validation Score: {sum(self.validation_scores) / len(self.validation_scores):,.5f}")

        self.feature_importance = pd.concat(self.feature_importance, axis=0)

        return clf_list, oof, prediction, self.feature_importance

    def do_adversarial_valid_kfold(self, model_conf, n_splits=2):
        sp = Splitter()
        target = "is_test"
        split_x = self.train["installation_id"]
        split_y = self.train[target]
        seed = 773
        sp.get_kfold_idx(split_x, split_y, seed, n_cv=n_splits, stratified=True, pref="adv")

        target_length = 1
        oof: ndarray = np.zeros(self.train.shape[0])
        prediction = np.zeros(self.test.shape[0])

        clf_list = []

        self.logger.log(logging.DEBUG, "[train cols] " + "-"*50)
        self.logger.log(logging.DEBUG, model_conf["train_cols"])
        self.validation_scores = []

        for i, (trn_idx, val_idx) in enumerate(sp.idx_list):
            self.logger.log(logging.DEBUG, "-" * 60)
            self.logger.log(logging.DEBUG, f"start training: {i}")

            with timer(f"fold {i}", self.logger):
                train_df, valid_df = self.train.loc[trn_idx], self.train.loc[val_idx]
                model = self.generate_model(model_conf)
                clf, fold_oof, feature_importance_df = model.train(train_df, valid_df, self.logger)

                # calc validation score using clf.best_iteration_
                fold_val_score = get_val_score(valid_df[target], fold_oof)
                self.validation_scores.append(fold_val_score)
                self.logger.log(logging.DEBUG, f"fold_val_score: {fold_val_score:,.5f}")

                clf_list.append(clf)
                oof[val_idx] = fold_oof

                feature_importance_df["fold"] = i
                self.feature_importance.append(feature_importance_df)

        self.logger.log(logging.DEBUG,
                        f"Total Validation Score: {sum(self.validation_scores) / len(self.validation_scores):,.5f}")

        oof = np.expm1(oof)
        self.train["pred_y"] = oof
        self.feature_importance = pd.concat(self.feature_importance, axis=0)

        return clf_list, oof, prediction, self.feature_importance


class Splitter():
    def __init__(self):
        self.save_dir = os.path.join(os.path.dirname("__file__"), "../data/valid_idx")
        self.idx_list = []
        os.makedirs(self.save_dir, exist_ok=True)

    def get_kfold_idx(self, split_x, split_y, seed, n_cv=5, stratified=True, group=False, pref=""):
        if group is False:
            if stratified:
                self.folds = StratifiedKFold(n_splits=n_cv, shuffle=False, random_state=seed)
            else:
                self.folds = KFold(n_splits=n_cv, shuffle=True, random_state=seed)

            for i, (trn_, val_) in enumerate(self.folds.split(split_x, split_y)):
                self.idx_list.append([trn_, val_])
                idx_trn_path = os.path.join(self.save_dir, "trn_idx_{}{}_{}.npy".format(pref, i, seed))
                idx_val_path = os.path.join(self.save_dir, "val_idx_{}{}_{}.npy".format(pref, i, seed))
                np.save(idx_trn_path, trn_)
                np.save(idx_val_path, val_)

            return self.idx_list
    
        else:
            groups = split_x
            self.folds = GroupKFold(n_splits=n_cv)
            for i, (trn_, val_) in enumerate(self.folds.split(split_x, split_y, groups)):
                self.idx_list.append([trn_, val_])
                idx_trn_path = os.path.join(self.save_dir, "trn_idx_{}{}_{}.npy".format(pref, i, seed))
                idx_val_path = os.path.join(self.save_dir, "val_idx_{}{}_{}.npy".format(pref, i, seed))
                np.save(idx_trn_path, trn_)
                np.save(idx_val_path, val_)

            return self.idx_list                

# feature engineering

## sample feature 

In [ ]:
class sample_feature(Features):
    """sample feature
    """
    def __int__(self, params, logger):
        super().__init__(params, logger)

    def calc_feature(self, org_train, org_test):
        
        if self.datatype == "train":
            df = org_train
        else:
            df = org_test
            
        feat_df = df.groupby("installation_id")["title"].count().reset_index()
        self.format_and_save_feats(feat_df)
        
        return feat_df

## base kernel features

In [ ]:
class KernelBasics(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def mapping_codes(self, org_train, org_test):
        self.all_activities = set(org_train["title"].unique()).union(
            set(org_test["title"].unique()))
        self.all_event_codes = set(org_train["event_code"].unique()).union(
            org_test["event_code"].unique())
        
        # convert activities <=> int
        self.activities_map = dict(
            zip(self.all_activities, np.arange(len(self.all_activities)))) # activity title => int 
        self.inverse_activities_map = dict(
            zip(np.arange(len(self.all_activities)), self.all_activities)) # int => activity title 
        
        # convert win_code <=> int 
        win_code = dict(
            zip(activities_map.values(),
                (4100 * np.ones(len(activities_map))).astype(int)))
        win_code[activities_map["Bird Measurer (Assessment)"]] = 4110
        
        self.win_code = win_code
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("true")), 'num_correct'] = 1
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("false")), 'num_incorrect'] = 1    
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")
        self.format_and_save_feats(ret)
        
        return ret
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        # initialize user activity 
        # 1. time spent 
        # 2. event count 
        # 3. session count 
        
        # sessionごとのplaytimeを算出
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある

        pv = pd.pivot_table(df, index=["gs_max_time", "game_session", "type"],  
                            columns="title", 
                            values="game_time", 
                            aggfunc="max").fillna(0)
        
        # 時刻順に並ぶことを保証する
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True)
        
        cum_cols = [c for c in list(pv.columns) if c not in ["type", "game_session", "gs_max_time"]]
        pv[cum_cols] = (pv[cum_cols].cumsum() // 1000).astype("int32")
        pv[cum_cols] = pv[cum_cols].shift(1) # 直前までのplaytimeを取得する
        
        ins_id = df.installation_id.values[0]
        
        # assessmentのrowに限定して抽出する
        if self.datatype == "train":
            # 正解ラベル/num_corrects を得るためtrain labelsとmerge
            pv = pd.merge(pv, self.train_labels[self.train_labels.installation_id == ins_id], how="inner", on="game_session")
        else:
            # calc num corrects 
            num_c_df = df.loc[df.type == "Assessment"].groupby(["installation_id","game_session"])[["num_correct", "num_incorrect"]].sum().fillna(0).reset_index()
            pv = pd.merge(pv, num_c_df, how="left", on="game_session")
            
        gc.collect()
        
        # 直前までの正解状況を集計
        pv["prev_num_corrects"] = pv["num_correct"].shift(1).fillna(0)
        pv["prev_cumnum_c"] = pv["prev_num_corrects"].cumsum()
        pv["prev_num_incorrects"] = pv["num_incorrect"].shift(1).fillna(0)
        pv["prev_cumnum_inc"] = pv["prev_num_incorrects"].cumsum()

        pv["cum_accuracy"] = (pv["prev_cumnum_c"] / 
                                     (pv["prev_cumnum_c"] + pv["prev_cumnum_inc"])).fillna(0)
        
        del pv["num_correct"], pv["num_incorrect"], pv["gs_max_time"], pv["prev_num_corrects"], pv["prev_num_incorrects"]
        gc.collect()
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])

        return pv
    
    
class KernelBasics2(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def mapping_codes(self, org_train, org_test):
        self.all_activities = set(org_train["title"].unique()).union(
            set(org_test["title"].unique()))
        self.all_event_codes = set(org_train["event_code"].unique()).union(
            org_test["event_code"].unique())
        
        # convert activities <=> int
        self.activities_map = dict(
            zip(self.all_activities, np.arange(len(self.all_activities)))) # activity title => int 
        self.inverse_activities_map = dict(
            zip(np.arange(len(self.all_activities)), self.all_activities)) # int => activity title 
        
        # convert win_code <=> int 
        win_code = dict(
            zip(activities_map.values(),
                (4100 * np.ones(len(activities_map))).astype(int)))
        win_code[activities_map["Bird Measurer (Assessment)"]] = 4110
        
        self.win_code = win_code
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            df = df.loc[df.installation_id.isin(self.train_labels.installation_id.unique())]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
            c_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true")))

            inc_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false")))
            
            df.loc[c_ass_idx, 'num_correct'] = 1
            df.loc[inc_ass_idx, 'num_incorrect'] = 1    
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")
        self.format_and_save_feats(ret)
        
        use_cols = [c for c in list(ret.columns) if "Assessment" not in c]
        
        return ret[use_cols]
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        # initialize user activity 
        # 1. time spent 
        # 2. event count 
        # 3. session count 
        
        # sessionごとのplaytimeを算出
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある

        pv = pd.pivot_table(df, index=["gs_max_time", "game_session", "type"],  
                            columns="title", 
                            values="game_time", 
                            aggfunc="max").fillna(0)
        
        # 時刻順に並ぶことを保証する
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True)
        
        cum_cols = [c for c in list(pv.columns) if c not in ["type", "game_session", "gs_max_time"]]
        pv[cum_cols] = (pv[cum_cols].cumsum() // 1000).astype("int32")
        pv[cum_cols] = pv[cum_cols].shift(1) # 直前までのplaytimeを取得する
        
        ins_id = df.installation_id.values[0]
        
        # assessmentのrowに限定して抽出する
        if self.datatype == "train":
            # 正解ラベル/num_corrects を得るためtrain labelsとmerge
            pv = pd.merge(pv, self.train_labels[self.train_labels.installation_id == ins_id], how="inner", on="game_session")
        else:
            # calc num corrects 
            num_c_df = df.loc[df.type == "Assessment"].groupby(["installation_id","game_session"])[["num_correct", "num_incorrect"]].sum().fillna(0).reset_index()
            pv = pd.merge(pv, num_c_df, how="left", on="game_session")
            
        gc.collect()
        
        # 直前までの正解状況を集計
        pv["prev_num_corrects"] = pv["num_correct"].shift(1).fillna(0)
        pv["prev_cumnum_c"] = pv["prev_num_corrects"].cumsum()
        pv["prev_num_incorrects"] = pv["num_incorrect"].shift(1).fillna(0)
        pv["prev_cumnum_inc"] = pv["prev_num_incorrects"].cumsum()

        pv["cum_accuracy"] = (pv["prev_cumnum_c"] / 
                                     (pv["prev_cumnum_c"] + pv["prev_cumnum_inc"])).fillna(0)
        
        del pv["num_correct"], pv["num_incorrect"]
        gc.collect()
        
        pv = self.get_acc_group(pv)        
        del pv["gs_max_time"]
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])

        return pv
    
    def get_acc_group(self, pv):
        def calc_accuracy_group(row):
            if row["prev_num_incorrects"] + row["prev_num_corrects"] > 0:
                acc = row["prev_num_corrects"] / (row["prev_num_incorrects"] + row["prev_num_corrects"])
                if acc == 0:
                    return 0
                elif acc == 1:
                    return 3
                elif acc == 0.5:
                    return 2
                else:
                    return 1
            else:
                return -99

        pv["acc_group"] = pv.apply(calc_accuracy_group, axis=1)
        acc_pv = pd.pivot_table(pv[["gs_max_time", "installation_id", "game_session", "acc_group"]], index=["gs_max_time", "game_session"], columns="acc_group", values="installation_id", aggfunc="count").reset_index().fillna(0)
        
        del pv["acc_group"]
        
        acc_columns = {}
        for col in acc_pv.columns:
            if col in [-99, 0, 1, 2, 3]:
                acc_columns[col] = "prev_acc_gr_" + str(col)
                acc_pv[f"accum_acc_gr_{col}" ] = acc_pv[col].cumsum()

        acc_pv.rename(columns=acc_columns, inplace=True)
        del acc_pv["gs_max_time"]
        pv = pd.merge(pv, acc_pv, on="game_session", how="left")    

        return pv


## EventCount

In [ ]:
class EventCount(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("true")), 'num_correct'] = 1
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("false")), 'num_incorrect'] = 1    
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")
#         self.format_and_save_feats(ret)

        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある

        pv = pd.pivot_table(df, index=["installation_id", "gs_max_time", "game_session", "type"],  
                            columns="event_code", 
                            values="timestamp", 
                            aggfunc="count").fillna(0)

        # 時刻順に並ぶことを保証する
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True)

        cum_cols = [c for c in list(pv.columns) if c not in ["installation_id", "type", "game_session", "gs_max_time"]]
        pv[cum_cols] = pv[cum_cols].cumsum().shift(1).fillna(0).astype("int32")
        
        pv = pv.loc[pv["type"] == "Assessment"] # assessment だけとればOK
        
        rename_dict = {}
        for c in cum_cols:
            rename_dict[c] = "ev_cnt" + str(c)     
        pv.rename(columns=rename_dict, inplace=True)
        pv.reset_index(inplace=True, drop=True)
        
        del pv["gs_max_time"], pv["type"]
        gc.collect()
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])

        return pv
class EventCount2(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("true")), 'num_correct'] = 1
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("false")), 'num_incorrect'] = 1    
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")
#         self.format_and_save_feats(ret)

        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある
        
        pv = pd.pivot_table(df, index=["installation_id", "gs_max_time", "game_session", "type"],  
                            columns="event_id", 
                            values="timestamp", 
                            aggfunc="count").fillna(0)

        # 時刻順に並ぶことを保証する
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True)

        cum_cols = [c for c in list(pv.columns) if c not in ["installation_id", "type", "game_session", "gs_max_time"]]
        pv[cum_cols] = pv[cum_cols].cumsum().shift(1).fillna(0).astype("int32")
        
        pv = pv.loc[pv["type"] == "Assessment"] # assessment だけとればOK
        
        rename_dict = {}
        for c in cum_cols:
            rename_dict[c] = "ev_cnt" + str(c)     
        pv.rename(columns=rename_dict, inplace=True)
        pv.reset_index(inplace=True, drop=True)
        
        del pv["gs_max_time"], pv["type"]
        gc.collect()
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])

        return pv


## Worldcount

In [ ]:
class Worldcount(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
                            
        ret = applyParallel(df.groupby("installation_id"), self.count_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["game_session","installation_id","title","type"]]
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")

        use_cols = [c for c in list(ret.columns) if c not in ["title","type", "event_code", "gs_max_time"]]
        
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def count_sessions(self, df):
        world_cnt = self.sub_count_sessions(df, ["world"], "wrd_cnt_")
        world_type_cnt = self.sub_count_sessions(df, ["world", "type"], "wrd_type_cnt_")        
        title_type_cnt = self.sub_count_sessions(df, ["title", "type"], "title_type_cnt_")
        
        world_cnt = pd.merge(world_cnt, world_type_cnt, how="left", on=["installation_id", "game_session"])
        del world_type_cnt
        
        world_cnt = pd.merge(world_cnt, title_type_cnt, how="left", on=["installation_id", "game_session"])
        del title_type_cnt        
        
        return world_cnt
    
    def sub_count_sessions(self, df, group_columns, prefix):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """        
        assess_sessions = df[df.type == "Assessment"]["game_session"].unique()
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある

        pv = pd.pivot_table(df, index=["installation_id", "gs_max_time", "game_session"],  
                            columns=group_columns, 
                            values="timestamp", 
                            aggfunc="count").fillna(0)

        # 時刻順に並ぶことを保証する
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True)
        
        if len(group_columns) >= 2:
            pv.columns = [c[0] + "_" + c[1] for c in pv.columns] 
            pv.rename(columns={"installation_id_": "installation_id", 
                               "game_session_": "game_session", 
                               "gs_max_time_": "gs_max_time"
                              }, inplace=True)

        cum_cols = [c for c in list(pv.columns) if c not in ["installation_id", "game_session", "gs_max_time"]]
        pv[cum_cols] = pv[cum_cols].cumsum().shift(1).fillna(0).astype("int32")


        rename_dict = {}
        for c in cum_cols:
            rename_dict[c] = prefix + str(c)     
        pv.rename(columns=rename_dict, inplace=True)
        pv.reset_index(inplace=True, drop=True)
        
        del pv["gs_max_time"]
        gc.collect()
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])
        elif self.datatype =="train":
            pv = pv.loc[pv.game_session.isin(assess_sessions)]

        return pv


## SessionTime

In [ ]:
class SessionTime(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
                            
        ret = applyParallel(df.groupby("installation_id"), self.time_sessions)
        use_cols = [c for c in list(ret.columns) if c not in ["title","type", "world",
                                                              "event_code", "gs_max_time", "timestamp_max", "timestamp_min"]]
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def count_sessions(self, df):
        world_cnt = self.sub_count_sessions(df, ["world"], "wrd_cnt_")
        world_type_cnt = self.sub_count_sessions(df, ["world", "type"], "wrd_type_cnt_")        
        title_type_cnt = self.sub_count_sessions(df, ["title", "type"], "title_type_cnt_")
        
        world_cnt = pd.merge(world_cnt, world_type_cnt, how="left", on=["installation_id", "game_session"])
        del world_type_cnt
        
        world_cnt = pd.merge(world_cnt, title_type_cnt, how="left", on=["installation_id", "game_session"])
        del title_type_cnt        
        
        return world_cnt
    
    def time_sessions(self, ins_df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """        
        agg_dict = {
            "timestamp" : ["max", "min"]
        }
        duration_df = groupings(ins_df, ["installation_id", "world", "type", "game_session"], agg_dict).sort_values("timestamp_min", ascending=True).reset_index(drop=True)

        duration_df["prev_gs_duration"] = (duration_df["timestamp_max"] - duration_df["timestamp_min"]).shift(1).dt.total_seconds()
        duration_df["session_interval"] = (duration_df["timestamp_min"] - duration_df["timestamp_max"].shift(1)).dt.total_seconds()
        
        window = 25
        min_periods = 5
        for col in ["prev_gs_duration", "session_interval"]:
            duration_df[col + "rmean"] = duration_df[col].rolling(window=window, min_periods=min_periods).mean()
            duration_df[col + "rstd"] = duration_df[col].rolling(window=window, min_periods=min_periods).std()
            duration_df[col + "rmax"] = duration_df[col].rolling(window=window, min_periods=2).max()
            duration_df[col + "rmin"] = duration_df[col].rolling(window=window, min_periods=2).min()        
            
        duration_df = duration_df.loc[duration_df.type == "Assessment"]
        
        if self.datatype=="test":
            duration_df = pd.DataFrame([duration_df.iloc[-1, :]])

        return duration_df
    
    
class SessionTime2(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            df = df.loc[df.installation_id.isin(self.train_labels.installation_id.unique())]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
                            
        ret = applyParallel(df.groupby("installation_id"), self.time_sessions)
        use_cols = [c for c in list(ret.columns) if c not in ["title","type", "world",
                                                              "event_code", "gs_max_time", "timestamp_max", "timestamp_min"]]
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def time_sessions(self, ins_df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """        
        ### session feature for all "type"
        agg_dict = {
            "timestamp" : ["max", "min"]
        }
        duration_df = groupings(ins_df, ["installation_id", "world", "type", "game_session"], agg_dict).sort_values("timestamp_min", ascending=True).reset_index(drop=True)

        duration_df["prev_gs_duration"] = (duration_df["timestamp_max"] - duration_df["timestamp_min"]).shift(1).dt.total_seconds()
        duration_df["session_interval"] = (duration_df["timestamp_min"] - duration_df["timestamp_max"].shift(1)).dt.total_seconds()
        
        window = 5
        min_periods = 2
        for col in ["prev_gs_duration", "session_interval"]:
            duration_df[col + "rmean"] = duration_df[col].rolling(window=window, min_periods=min_periods).mean()
            duration_df[col + "rstd"] = duration_df[col].rolling(window=window, min_periods=min_periods).std()
            duration_df[col + "rmax"] = duration_df[col].rolling(window=window, min_periods=2).max()
            duration_df[col + "rmin"] = duration_df[col].rolling(window=window, min_periods=2).min()        
            
        duration_df = duration_df.loc[duration_df.type == "Assessment"]
        
        ### session feature for "assessments"        
        agg_dict = {
            "timestamp" : ["max", "min"]
        }
        ass_duration = groupings(ins_df, ["installation_id", "world", "type", "game_session"], agg_dict)
        ass_duration = ass_duration.loc[ass_duration.type == "Assessment"].sort_values("timestamp_min", ascending=True).reset_index(drop=True)

        ass_duration["prev_ass_gs_duration"] = (ass_duration["timestamp_max"] - ass_duration["timestamp_min"]).shift(1).dt.total_seconds()
        ass_duration["ass_session_interval"] = (ass_duration["timestamp_min"] - ass_duration["timestamp_max"].shift(1)).dt.total_seconds()

        window = 5
        min_periods = 1
        for col in ["prev_ass_gs_duration", "ass_session_interval"]:
            ass_duration[col + "_rmean"] = ass_duration[col].rolling(window=window, min_periods=min_periods).mean()
            ass_duration[col + "_rstd"] = ass_duration[col].rolling(window=window, min_periods=min_periods).std()
            ass_duration[col + "_rmax"] = ass_duration[col].rolling(window=window, min_periods=1).max()
            ass_duration[col + "_rmin"] = ass_duration[col].rolling(window=window, min_periods=1).min()        
        
        ass_cols = [c for c in list(ass_duration.columns) if c not in ['installation_id', 'world', 'type','timestamp_max','timestamp_min']]
        
        duration_df = pd.merge(duration_df, ass_duration[ass_cols], how="left", on="game_session")
        
        if self.datatype=="test":
            duration_df = pd.DataFrame([duration_df.iloc[-1, :]])

        return duration_df    

## EncodingTitles

In [ ]:
class EncodingTitles(Features):
    """Event count in only Assessments
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def get_encoder(self, org_train, org_test):
        self.all_activities = set(org_train["title"].unique()).union(
            set(org_test["title"].unique()))
        self.all_event_codes = set(org_train["event_code"].unique()).union(
            org_test["event_code"].unique())
        self.activities_map = dict(
            zip(self.all_activities, np.arange(len(self.all_activities))))
        self.inverse_activities_map = dict(
            zip(np.arange(len(self.all_activities)), self.all_activities))
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("true")), 'num_correct'] = 1
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("false")), 'num_incorrect'] = 1    
            df = org_test
        
        # get encodings informations
        self.get_encoder(org_train, org_test)
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        
        self.format_and_save_feats(ret[use_cols])
        return ret[use_cols]
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        df["title_enc"] = df["title"].map(self.activities_map)
        df = df.loc[df.type=="Assessment"][["installation_id", "game_session", "title_enc"]].drop_duplicates().reset_index(drop=True)
        
        if self.datatype=="test":
            df = pd.DataFrame([df.iloc[-1, :]])

        return df

## PrevAssessResult

In [ ]:
class PrevAssessResult(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("true")), 'num_correct'] = 1
            org_test.loc[(org_test.event_code.isin([4100, 4110])) & (org_test["event_data"].str.contains("false")), 'num_incorrect'] = 1    
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        ret[ret_col] = ret[ret_col].fillna(0).astype("int32")
#         self.format_and_save_feats(ret)

        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
    
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        # 単純なactivity count
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある
        pv = pd.pivot_table(df, index=["installation_id", "gs_max_time", "game_session", "type"],  
                            columns="title", 
                            values="timestamp", 
                            aggfunc="count").fillna(0)


        assess_col = [c for c in list(pv.columns) if "Assessment" in c]
        pv = pv[assess_col]
        pv.reset_index(inplace=True)

        rename_dict = {}
        new_cols = []
        
        cnt_pref = "assess_cnt_"
        for c in assess_col:
            rename_dict[c] = cnt_pref + str(c)     

        pv = pv.loc[pv.type=="Assessment"].reset_index(drop=True)
        pv.sort_values("gs_max_time", ascending=True, inplace=True)
        pv.reset_index(inplace=True, drop=True)
        pv[assess_col] = pv[assess_col].shift(1).fillna(0)
        pv.rename(columns=rename_dict, inplace=True)
        
        for c in assess_col:
            pv["accum" + cnt_pref + str(c)] = pv[cnt_pref + str(c)].cumsum()

        del pv["gs_max_time"], pv["type"]

        return pv

## PrevAssessAcc

In [ ]:
def assess_history(gr_df):
    gr_df = gr_df.sort_values("gs_max_time", ascending=True)

    gr_df["as_acc_c_num"] = gr_df["num_correct"].cumsum()
    gr_df["as_acc_inc_num"] = gr_df["num_incorrect"].cumsum()
    gr_df["as_prev_acc"] = gr_df["num_correct"] / (gr_df["num_correct"] + gr_df["num_incorrect"])
    gr_df["as_cum_acc"] = gr_df["as_acc_c_num"] / (gr_df["as_acc_c_num"] + gr_df["as_acc_inc_num"])

    shift_col = ["num_correct", "num_incorrect", "as_acc_c_num", "as_acc_inc_num", "as_prev_acc", "as_cum_acc"]
    gr_df[shift_col] = gr_df[shift_col].shift(1).fillna(-99)

    return gr_df    

class PrevAssessAcc(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            assess_user = df.loc[df.type == "Assessment"].installation_id.unique()
            df = df.loc[df.installation_id.isin(assess_user)]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        ret[ret_col] = ret[ret_col].fillna(0)
#         self.format_and_save_feats(ret)

        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
        
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        # 単純なactivity count
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある
        
        c_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true")))

        inc_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false")))

        df.loc[c_ass_idx, 'num_correct'] = 1
        df.loc[inc_ass_idx, 'num_incorrect'] = 1

        df["num_correct"].fillna(0, inplace=True)
        df["num_incorrect"].fillna(0, inplace=True)

        df = df.loc[(df.type =="Assessment")]
        
        df = df.groupby(["installation_id", "game_session", "gs_max_time", "title"])[["num_correct", "num_incorrect"]].sum().reset_index()
               
        df = df.groupby("title").apply(assess_history)
        df = df.sort_values("gs_max_time", ascending=True)

        del df["title"], df["gs_max_time"]
        
        if self.datatype=="test":
            df = pd.DataFrame([df.iloc[-1, :]])
       
        return df
    


## PrevAssessAccByTitle

In [ ]:
class PrevAssessAccByTitle(Features):
    """kernel features revised
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            df = df.loc[df.installation_id.isin(self.train_labels.installation_id.unique())]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        ret[ret_col] = ret[ret_col].fillna(0)
#         self.format_and_save_feats(ret)

        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
        
    def ins_id_sessions(self, df):
        """session当該session直前までのactivityを示す
        Args:
            df: df grouped by installation_id 
        """
        # 単純なactivity count
        df["gs_max_time"] = df.groupby("game_session")["timestamp"].transform("max") # gs_max_timeでsortする必要がある
        
        c_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("true")))

        inc_ass_idx = ((df.event_code == 4100) 
                          & (df.title != "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false"))) | \
                         ((df.event_code == 4110) 
                          & (df.title == "Bird Measurer (Assessment)") 
                          & (df["event_data"].str.contains("false")))

        df.loc[c_ass_idx, 'num_correct'] = 1
        df.loc[inc_ass_idx, 'num_incorrect'] = 1

        df["num_correct"].fillna(0, inplace=True)
        df["num_incorrect"].fillna(0, inplace=True)

        df = df.loc[(df.type =="Assessment")]
        
        pv = pd.pivot_table(df, index=["installation_id", "game_session", "gs_max_time"], columns="title", values=["num_correct", "num_incorrect"], aggfunc="sum").reset_index().sort_values("gs_max_time")

        pv.columns = [c[0] + "_" + c[1] if c[1] != "" else c[0] for c in list(pv.columns)]
        pv_num_cols = [c for c in list(pv.columns) if "correct" in c]

        cum_cols = ["cum_" + c for c in pv_num_cols] # 累積列

        pv_cum_corr_cols = [c for c in cum_cols if "cum_num_correct_" in c] # correct 列のみ 
        pv_cum_incorr_cols = [c for c in cum_cols if "cum_num_incorrect_" in c] # incorrect 列
        pv_cum_acc_cols = ["cum_acc_" + re.sub('num_incorrect_', '', c) for c in pv_cum_incorr_cols]

        pv.reset_index(drop=True, inplace=True)

        pv[pv_num_cols] = pv[pv_num_cols].shift(1).fillna(0)
        pv[cum_cols] = pv[pv_num_cols].cumsum()

        pv[pv_cum_acc_cols] = pd.DataFrame(pv[pv_cum_corr_cols].values / (pv[pv_cum_corr_cols].values + pv[pv_cum_incorr_cols].values))

        del pv["gs_max_time"]
        
        if self.datatype=="test":
            pv = pd.DataFrame([pv.iloc[-1, :]])
       
        return pv
    

## GameDurMiss

In [ ]:
def game_duration(val):
    val = json.loads(val)
    duration = val["duration"]
    g_misses = val["misses"]
    
    return [duration, g_misses]

def world_cum_duration_calc(world_df):
    # duration / missを抽出
    wg_df = world_df[(world_df.event_code == 2030) & (world_df.type=="Game")]
    du_miss = np.array(wg_df["event_data"].apply(game_duration).tolist())
    try:
        wg_df["duration"] = du_miss[:, 0]
        wg_df["misses"] = du_miss[:, 1]
    except:
        wg_df["duration"] = np.nan
        wg_df["misses"] = np.nan

    del du_miss

    aggs = {
        "duration": ["min", "mean", "max", "std", "count"],
        "misses": ["min", "mean", "max", "std"],
    }

    game_cums = groupings(wg_df, ["game_session", "gs_max_time", "world"], aggs, "g_")

    del wg_df
    gc.collect()

    # 累積を計算
    game_cums = game_cums.sort_values("gs_max_time").reset_index(drop=True)

    num_cols = [c for c in list(game_cums.columns) if c not in ["game_session", "gs_max_time", "world"] ]
    cum_mean_cols = ["mean_" + c for c in num_cols]

    game_cums[cum_mean_cols] = game_cums[num_cols].cumsum()
    game_cums["cumnum"] = (game_cums.index + 1).values
    game_cums[cum_mean_cols] /= game_cums["cumnum"].values.reshape((-1, 1))

    game_cums[["game_session", "gs_max_time", "world"] + cum_mean_cols]

    # 直前のgameまでの累積結果をmergeする
    game_ass_uni = world_df[["world", "game_session", "type", "installation_id","gs_max_time"]].drop_duplicates().sort_values("gs_max_time").reset_index(drop=True)

    game_ass_uni = pd.merge(game_ass_uni, game_cums, how="left", on=["game_session", "gs_max_time", "world"]).fillna(method="ffill")
    game_ass_uni = game_ass_uni.loc[game_ass_uni.type=="Assessment"]

    return game_ass_uni

class GameDurMiss(Features):
    """assessment 直前までのgameのプレイ状況を取得する
    """
    def __init__(self, train_labels, params, logger=None):        
        super().__init__(params, logger=logger)
        self.train_labels = train_labels        
        
    def calc_feature(self, org_train, org_test):                
        if self.datatype == "train":
            df = org_train
            df = df.loc[df.installation_id.isin(self.train_labels.installation_id.unique())]
        else:
            # 直前までのnum_correct/incorrectを取得する
            df = org_test
        
        ret = applyParallel(df.groupby("installation_id"), self.ins_id_sessions)
        ret_col = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy",
                                                             "game_session","installation_id","title",
                                                             "type"
                                                            ]]
        
        use_cols = [c for c in list(ret.columns) if c not in ["accuracy","accuracy_group","cum_accuracy","title",
                                                             "type", "event_code", "gs_max_time"
                                                            ]]
        self.format_and_save_feats(ret[use_cols])
        
        return ret[use_cols]
        
    def ins_id_sessions(self, df):
        """assessment 直前までのgameのプレイ状況を取得する
        Args:
            df: df grouped by installation_id 
        """
        # 単純なactivity count
        gs_game_ass = df.loc[((df.event_code == 2030) & (df.type=="Game")) | (df.type=="Assessment")]
        gs_game_ass["gs_max_time"] = gs_game_ass.groupby("game_session")["timestamp"].transform("max")        
        
        game_ass_uni = gs_game_ass.groupby("world").apply(world_cum_duration_calc).reset_index(drop=True).sort_values("gs_max_time")
        
        del gs_game_ass
        
        if self.datatype=="test":
            game_ass_uni = pd.DataFrame([game_ass_uni.iloc[-1, :]])
       
        return game_ass_uni


# Optimize Rounder

In [ ]:
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

# preprocess 

## add features

In [ ]:
def preprocess_dfs(use_features, is_local=False, logger=None, debug=True):
    # read dataframes     
    with timer("read datasets"):
        if debug: 
            nrows=200000 
        else: nrows=None
            
        sub = pd.read_csv(base_path + '/sample_submission.csv')
        
        if is_local:
            org_train = pickle_load("../input/train.pkl")
            org_test = pickle_load("../input/test.pkl")
        else:
            org_train = pd.read_csv(base_path + "/train.csv", nrows=nrows)
            org_test = pd.read_csv(base_path + "/test.csv", nrows=nrows)
            
        org_train = memory_reducer(org_train, verbose=True)
        org_test = org_test[org_test.installation_id.isin(sub.installation_id)]
        org_test.sort_values(['installation_id', 'timestamp'], inplace=True)
        org_test.reset_index(inplace=True)
        org_test = memory_reducer(org_test, verbose=True)
        
        train_labels = pd.read_csv(base_path + "/train_labels.csv", nrows=nrows)
        specs = pd.read_csv(base_path + "/specs.csv", nrows=nrows)

    # basic preprocess
    org_train["timestamp"] = pd.to_datetime(org_train["timestamp"])
    org_test["timestamp"] = pd.to_datetime(org_test["timestamp"])
    
    with timer("merging features"):
        train_df = add_features(use_features, org_train, org_test, train_labels, specs, datatype="train", is_local=is_local, logger=None)
        train_df = train_df.reset_index(drop=True)
        test_df = add_features(use_features, org_train, org_test, train_labels, specs, datatype="test", is_local=is_local, logger=None)
        test_df = test_df.reset_index(drop=True)
    
#     df = pd.concat([df, feat_df], axis=1)
    print("preprocess done!!")

    return train_df, test_df


def feature_maker(feat_cls, is_overwrite, org_train, org_test, train_labels, params, logger, is_local):
    """featureの読み込み
    """
    feat_ = feat_cls(train_labels, params, logger)
    feat_name = feat_.name
    datatype = feat_.datatype
    feature_dir = os.path.join(os.path.dirname("__file__"), "../feature")
    feature_path = Path(feature_dir) / f"{datatype}" / f"{feat_name}.pkl"
    
    if os.path.exists(feature_path) and is_overwrite is False:
        f_df = pickle_load(feature_path)
    else:
        f_df = feat_.feature_extract(org_train, org_test)
    
    return f_df


def add_features(use_features, org_train, org_test, train_labels, specs, datatype, is_local=False, logger=None):
    # 都度計算する
    feat_params = {
        "datatype": datatype,
        "debug": True,
        "is_overwrite": True,
    }

    # base feature
    base_feat = KernelBasics2(train_labels, feat_params, logger)
    feature_dir = os.path.join(os.path.dirname("__file__"), "../feature")
    feature_path = Path(feature_dir) / f"{datatype}" / f"{base_feat.name}.pkl"
    
    if os.path.exists(feature_path):
        feat_df = pickle_load(feature_path)
    else:
        feat_df = base_feat.feature_extract(org_train, org_test)

    # add event_counts
    for name, feat_condition in use_features.items():
        feat_cls = feat_condition[0]
        is_overwrite = feat_condition[1]
        
        f_df = feature_maker(feat_cls, is_overwrite, org_train, org_test, train_labels, feat_params, logger, is_local)
        feat_df = pd.merge(feat_df, f_df, how="left", on =["installation_id", "game_session"])
        del f_df

    return feat_df


# training


In [ ]:
use_feature = {
    "EventCount": [EventCount, True], # class, is_overwrite
    "EventCount2": [EventCount2, True], # class, is_overwrite
    "Worldcount": [Worldcount, True],
    "SessionTime": [SessionTime2, True],
#     "AssessEventCount": [AssessEventCount, False],
    "EncodingTitles": [EncodingTitles, True],
#     "PrevAssessResult":[PrevAssessResult, True],
#     "PrevAssessAcc": [PrevAssessAcc, True],
    "PrevAssessAccByTitle": [PrevAssessAccByTitle, True]
}

is_local = False

if is_local:
    base_path = "../input" # at local
    train_df, test_df = preprocess_dfs(use_feature, is_local=is_local, logger=None, debug=False)
    
else:
    sub = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')
    base_path = '/kaggle/input/data-science-bowl-2019' # at kaggle kernel
    if len(sub)==1000:
        sub.to_csv('submission.csv', index=False)
        exit(0)
    else:
        train_df, test_df = preprocess_dfs(use_feature, is_local=is_local, logger=None, debug=False)

## set configs

In [ ]:
exp_name = "suga_001_add_eventidcnt"
logger, log_path = log_output(exp_name)

train_small_dataset = False
is_debug = True

# train_params = {
#     'learning_rate': 0.01,
#     'bagging_fraction': 0.90,
#     'feature_fraction': 0.85,
#     'max_depth': 5,
#     'lambda_l1': 0.7,
#     'lambda_l2': 0.7,
#     'metric': 'multiclass',
#     'objective': 'multiclass',
#     'num_classes': 4,
#     'random_state': 773,
#     "n_estimators": 3000    

# }

train_params = {
     'learning_rate': 0.01,
     'boosting_type': 'gbdt',
     'objective': 'regression',
     'metric': 'rmse',
    'num_leaves':  64,
     'bagging_fraction': 0.9,
     'bagging_freq': 1,
     'feature_fraction': 0.7,
     'max_depth': -1,
     'lambda_l1': 0.2,
     'lambda_l2': 0.4,
     'seed': 19930802,
     'n_estimators': 100000
}

bad_feats = [
    'prev_gs_duration', 'session_intervalrmin', 'session_intervalrstd', 'session_intervalrmax', 'session_interval', 'accum_acc_gr_-99',
    'session_intervalrmean', 'ass_session_interval', 'prev_gs_durationrmean', 'prev_gs_durationrmax',
    'ev_cnt4070', 'prev_gs_durationrstd', 'mean_g_duration_meaan', 'ev_cnt3010', 'g_duration_std', 'ev_cnt4030', 'ev_cnt3110',
    'g_duration_mean', 'meaan_g_duration_min', 'ass_session_interval_rmin', 'accum_acc_gr_3', 'g_duration_min', 'mean_g_duraation_std'
]
no_use_cols = [
    "accuracy",
    "accuracy_group",
    "game_session",
    "installation_id",
    "title", 
    "type",
    "world",
    "pred_y"
] + list(set(train_df.columns) - set(test_df.columns)) + bad_feats


train_cols = [c for c in list(train_df.columns) if c not in no_use_cols]
    
print(f"train_df shape: {train_df.shape}")
print(train_cols)

cat_cols = [
            ]


# logger.log(logging.DEBUG, f"categorical cols: {cat_cols}")

target = "accuracy_group"

model_conf = {
    "predict_type": "regressor",
    "train_params": train_params,
    "train_cols": train_cols,
    "cat_cols": cat_cols,
    "target": target,
    "is_debug": is_debug,
}

validation_param = {
    "model_name": "LGBM",
}

exp_conf = {
    "train_small_dataset": False,
    "use_feature": {
        "sample": True
    },
    "train_params": train_params,
    "exp_name": exp_name
}

In [ ]:
v = Validation(validation_param, exp_conf, train_df, test_df, logger)
clf, oof, prediction, feature_importance = v.do_valid_kfold(model_conf)

In [ ]:
test_pred = prediction.copy()

In [ ]:
optR = OptimizedRounder()
optR.fit(oof, train_df[target])
coefficients = optR.coefficients()

opt_preds = optR.predict(oof, coefficients)
qwk(train_df[target], opt_preds)


In [ ]:
pd.Series(oof).hist(bins=15)

In [ ]:
pd.Series(prediction).hist(bins=50)

In [ ]:
coefficients

prediction = optR.predict(prediction, coefficients)

In [ ]:
feature_importance.groupby("feature")["importance"].mean().reset_index().sort_values("importance", ascending=False).head(50)

In [ ]:

sub_df =  pd.read_csv(base_path + "/sample_submission.csv")
# prediction.shape

In [ ]:
sub_df["accuracy_group"] = prediction
# .argmax(axis = 1)
sub_df["accuracy_group"].value_counts(normalize=True)
sub_df.to_csv('submission.csv', index=False)